# Modelo de Recomendacion Item-Item

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
# Cargar el conjunto de datos
df_juegos = pd.read_parquet("C:\\Users\\Gary Alexander Bean\\Desktop\\Proyecto-Individual-1-MLOps\\Datasets\\ArchivosPARQUET\\steam_games_limpio.parquet")

In [3]:
# Seleccionar las columnas necesarias
df_juegos_seleccionados = df_juegos.loc[:, ["specs", "item_id", "name"]]

In [4]:
# Convertir 'item_id' a tipo entero
df_juegos_seleccionados["item_id"] = df_juegos_seleccionados["item_id"].astype(int)

# Limpiar la columna 'specs'
df_juegos_seleccionados['specs'] = df_juegos_seleccionados['specs'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace("'", ''))

## Creación de un Vectorizador de Texto (CountVectorizer)

### Propósito

El objetivo principal del CountVectorizer es transformar datos de texto, específicamente la columna 'specs' en este caso, en representaciones numéricas comprensibles para algoritmos de aprendizaje automático. Al convertir el texto en vectores numéricos, permitimos que los modelos puedan procesar y comprender la información textual de manera eficiente.

### Proceso

El CountVectorizer opera asignando un índice único a cada palabra distinta que aparece en el texto y luego contando la frecuencia de cada una de estas palabras en cada documento (en este contexto, cada juego). En esencia, cada documento se convierte en un vector, donde cada posición del vector corresponde a una palabra única y el valor en esa posición representa la frecuencia de esa palabra en el documento. Este enfoque nos permite capturar la estructura y el contenido del texto de manera cuantitativa, lo que resulta esencial para el análisis y la manipulación subsiguientes.

### Importancia

La representación vectorial resultante es crucial para una amplia gama de aplicaciones en procesamiento de lenguaje natural y aprendizaje automático. Por ejemplo, habilita la comparación de documentos mediante métricas de similitud, como el coseno, que operan directamente sobre vectores numéricos. Además, esta representación permite la aplicación de técnicas más avanzadas, como la clasificación de documentos, la agrupación de temas y el análisis de sentimientos, entre otros. En resumen, el CountVectorizer es una herramienta fundamental para convertir datos textuales en formatos que puedan ser aprovechados por modelos de aprendizaje automático, abriendo así las puertas a un análisis más profundo y significativo del texto.

In [5]:
# Crear un vectorizador de texto
vectorizador_texto = CountVectorizer()
vectores_texto = vectorizador_texto.fit_transform(df_juegos_seleccionados['specs']).toarray()


In [6]:
# Calcular la similitud del coseno entre vectores
similitud_cos = cosine_similarity(vectores_texto)

In [7]:
def recomendacion_juego(id_producto):
    """
    Genera una lista de juegos recomendados similares al juego dado su ID.

    Parámetros:
    - id_producto (int): El ID del juego para el cual se desean obtener recomendaciones.

    Retorna:
    - list: Una lista con los nombres de los juegos recomendados.
    """
    # Se busca el índice del juego en el DataFrame original
    indice_producto = df_juegos_seleccionados[df_juegos_seleccionados["item_id"] == id_producto].index[0]
    
    # Cálculo de similitudes
    distancias = similitud_cos[indice_producto]
    
    # Las distancias se ordenan de manera descendente, y se seleccionan los cinco juegos más similares (excluyendo el juego de entrada)
    lista_juegos_similares = sorted(list(enumerate(distancias)), reverse=True, key=lambda x: x[1])[1:6]
    
    # Lista de títulos recomendados
    juegos_recomendados = [df_juegos_seleccionados.iloc[i[0]]['name'] for i in lista_juegos_similares]
    
    return juegos_recomendados

In [9]:
# Ejemplo de uso:
id_producto = 761140
resultado = recomendacion_juego(id_producto)
print(resultado)

['弹炸人2222', 'Uncanny Islands', 'Beach Rules', 'Planetarium 2 - Zen Odyssey', 'The Warrior Of Treasures']
